In [1]:
# import random
# import pandas as pd

# # Константы
# NUM_BUSES = 8
# NUM_DRIVERS = 10
# NUM_8HR_DRIVERS = 5
# NUM_12HR_DRIVERS = NUM_DRIVERS - NUM_8HR_DRIVERS
# SHIFT_DURATIONS = {"8hr": 8, "12hr": 12}
# DAYS_OF_WEEK = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
# TIME_SLOTS = range(24)

# # Параметры генетического алгоритма
# POPULATION_SIZE = 50
# MAX_GENERATIONS = 100
# MUTATION_RATE = 0.1
# CROSSOVER_RATE = 0.8


# # Проверка, является ли день отдыха для водителя
# def is_rest_day(driver_id, day_idx):
#     if driver_id < NUM_8HR_DRIVERS:  # 8-часовые водители отдыхают на выходных
#         return DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]
#     else:  # 12-часовые водители отдыхают 2 дня после работы
#         return day_idx % 3 != 0


# # Генерация случайного расписания
# def generate_random_schedule():
#     schedule = []
#     for day_idx, day in enumerate(DAYS_OF_WEEK):
#         daily_schedule = []
#         for driver_id in range(NUM_DRIVERS):
#             if is_rest_day(driver_id, day_idx):
#                 daily_schedule.append({"driver": driver_id, "bus": None, "start": None, "end": None, "rest": True})
#             else:
#                 bus = random.randint(0, NUM_BUSES - 1)
#                 shift_type = "8hr" if driver_id < NUM_8HR_DRIVERS else "12hr"
#                 start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
#                 end_time = (start_time + SHIFT_DURATIONS[shift_type]) % 24
#                 daily_schedule.append({
#                     "driver": driver_id,
#                     "bus": bus,
#                     "start": start_time,
#                     "end": end_time,
#                     "rest": False
#                 })
#         schedule.append(daily_schedule)
#     return schedule


# # Вычисление фитнес-функции
# def fitness(schedule):
#     penalty = 0
#     bus_coverage = {day: {hour: [] for hour in TIME_SLOTS} for day in DAYS_OF_WEEK}

#     for day_idx, daily_schedule in enumerate(schedule):
#         for driver_schedule in daily_schedule:
#             if driver_schedule["rest"]:
#                 continue

#             bus = driver_schedule["bus"]
#             start, end = driver_schedule["start"], driver_schedule["end"]

#             if bus is not None:
#                 hours = range(start, end + 1 if end >= start else end + 24)
#                 for hour in hours:
#                     bus_coverage[DAYS_OF_WEEK[day_idx]][hour % 24].append(bus)

#             if driver_schedule["driver"] < NUM_8HR_DRIVERS and DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]:
#                 penalty += 10

#             if driver_schedule["driver"] >= NUM_12HR_DRIVERS:
#                 if day_idx > 0:
#                     prev_day = schedule[day_idx - 1]
#                     for prev_driver in prev_day:
#                         if prev_driver["driver"] == driver_schedule["driver"] and not prev_driver["rest"]:
#                             penalty += 10

#     for day, coverage in bus_coverage.items():
#         for hour, buses in coverage.items():
#             if len(set(buses)) < NUM_BUSES:
#                 penalty += 5 * (NUM_BUSES - len(set(buses)))

#     return -penalty


# # Кроссовер
# def crossover(parent1, parent2):
#     point = random.randint(0, len(parent1) - 1)
#     return parent1[:point] + parent2[point:]


# # Мутация
# def mutate(schedule):
#     day_idx = random.randint(0, len(schedule) - 1)
#     driver_idx = random.randint(0, NUM_DRIVERS - 1)
#     shift_type = "8hr" if driver_idx < NUM_8HR_DRIVERS else "12hr"
#     duration = SHIFT_DURATIONS[shift_type]
#     bus = random.randint(0, NUM_BUSES - 1)
#     start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
#     end_time = (start_time + duration) % 24
#     schedule[day_idx][driver_idx] = {
#         "driver": driver_idx,
#         "bus": bus,
#         "start": start_time,
#         "end": end_time,
#         "rest": False
#     }
#     return schedule

# # Генетический алгоритм
# def genetic_algorithm():
#     population = [generate_random_schedule() for _ in range(POPULATION_SIZE)]
#     for generation in range(MAX_GENERATIONS):
#         population.sort(key=fitness, reverse=True)
#         next_generation = population[:5]

#         while len(next_generation) < POPULATION_SIZE:
#             if random.random() < CROSSOVER_RATE:
#                 parent1, parent2 = random.sample(population[:20], 2)
#                 child = crossover(parent1, parent2)
#             else:
#                 child = random.choice(population[:20])

#             if random.random() < MUTATION_RATE:
#                 child = mutate(child)

#             next_generation.append(child)

#         population = next_generation
#         #print(f"Generation {generation + 1}: Best Fitness = {fitness(population[0])}")

#     return population[0]


# # Форматирование расписания в DataFrame
# def generate_schedule_dataframe(schedule):
#     days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
#     driver_labels = [f"Driver {d + 1}" for d in range(NUM_DRIVERS)]
#     shifts = ["8hr" if d < NUM_8HR_DRIVERS else "12hr" for d in range(NUM_DRIVERS)]

#     schedule_df = pd.DataFrame(index=driver_labels, columns=["Shift"] + days)
#     schedule_df["Shift"] = shifts

#     for day_idx, daily_schedule in enumerate(schedule):
#         for driver_schedule in daily_schedule:
#             driver = f"Driver {driver_schedule['driver'] + 1}"
#             if driver_schedule["rest"]:
#                 schedule_df.loc[driver, days[day_idx]] = "Rest"
#             else:
#                 bus = driver_schedule["bus"]
#                 start = driver_schedule["start"]
#                 end = driver_schedule["end"]
#                 schedule_df.loc[driver, days[day_idx]] = f"Bus {bus + 1} ({start:02d}:00 - {end:02d}:00)"

#     return schedule_df.sort_values(by="Shift", ascending=False)


# # Запуск
# best_schedule = genetic_algorithm()
# schedule_df = generate_schedule_dataframe(best_schedule)
# schedule_df

In [2]:
# from datetime import datetime, timedelta

# def is_peak_hour(current_time: datetime) -> bool:
#     """Check if the current time falls within peak hours."""
#     return (current_time.time() >= datetime.strptime("07:00", "%H:%M").time() and
#             current_time.time() < datetime.strptime("09:00", "%H:%M").time()) or \
#            (current_time.time() >= datetime.strptime("17:00", "%H:%M").time() and
#             current_time.time() < datetime.strptime("19:00", "%H:%M").time())

# def get_trip_duration(start_time: datetime, duration: int) -> int:
#     """Determine the trip duration based on peak hours."""
#     end_time = start_time + timedelta(minutes=duration)

#     # Check if the trip overlaps with peak hours
#     if is_peak_hour(start_time) or is_peak_hour(end_time):
#         return 105  # 105 minutes during peak hours
#     else:
#         return 100  # 100 minutes otherwise

# def generate_schedule(start_time: str, work_duration: int = 12, rest_duration: int = 10):
#     start = datetime.strptime(start_time, "%H:%M")
#     end_time = start + timedelta(hours=work_duration)

#     current_time = start
#     trips = []
#     trip_count = 0

#     while current_time < end_time:
#         # Get the trip duration based on current and end time
#         trip_duration = get_trip_duration(current_time, 100)  # Initial guess of 100 minutes

#         trip_start = current_time
#         trip_end = current_time + timedelta(minutes=trip_duration)

#         if trip_end > end_time:
#             break

#         trips.append(f"Trip {trip_count + 1}: {trip_start.strftime('%H:%M')} - {trip_end.strftime('%H:%M')}")
#         trip_count += 1
#         current_time = trip_end

#         if trip_count % 2 == 0:  # Rest after every two trips
#             rest_start = current_time
#             rest_end = current_time + timedelta(minutes=rest_duration)

#             if rest_end > end_time:
#                 break

#             trips.append(f"Rest: {rest_start.strftime('%H:%M')} - {rest_end.strftime('%H:%M')}")
#             current_time = rest_end

#     return trips

# print("\nDriver 1 ---- Bus 1\n")
# schedule = generate_schedule("00:00")
# for entry in schedule:
#     print(entry)

# print("\nDriver 2 ---- Bus 2\n")
# schedule = generate_schedule("03:20")
# for entry in schedule:
#     print(entry)


# print("\nDriver 3 ---- Bus 1\n")
# schedule = generate_schedule("13:15")
# for entry in schedule:
#     print(entry)

# print("\nDriver 4 ---- Bus 2\n")
# schedule = generate_schedule("13:30")
# for entry in schedule:
#     print(entry)

In [3]:
from datetime import datetime, timedelta

def is_peak_hour(current_time: datetime) -> bool:
    """Проверяет, попадает ли время в часы пик."""
    return (datetime.strptime("07:00", "%H:%M").time() <= current_time.time() < datetime.strptime("09:00", "%H:%M").time()) or \
           (datetime.strptime("17:00", "%H:%M").time() <= current_time.time() < datetime.strptime("19:00", "%H:%M").time())


def get_trip_duration(start_time: datetime, base_duration: int = 90) -> int:
    """
    Рассчитывает длительность поездки:
    - Увеличивает длительность на 5 минут, если поездка пересекает часы пик.
    """ 
    end_time = start_time + timedelta(minutes=base_duration)
    
    # Если поездка пересекает часы пик, увеличить длительность на 5 минут
    if is_peak_hour(start_time) or is_peak_hour(end_time):
        base_duration += 5

    return base_duration



def generate_schedule(start_time: str, work_duration: int = 12, rest_duration: int = 10):
    """
    Генерирует расписание с учетом:
    - Времени начала.
    - Рабочей смены.
    - Перерывов каждые 2 поездки.
    """
    start = datetime.strptime(start_time, "%H:%M")
    end_time = start + timedelta(hours=work_duration)

    current_time = start
    trips = []
    trip_count = 0

    while current_time < end_time:
        # Рассчитать длительность поездки
        trip_duration = get_trip_duration(current_time)
        trip_start = current_time
        trip_end = current_time + timedelta(minutes=trip_duration)

        if trip_end > end_time:
            break

        trips.append(f"Trip {trip_count + 1}: {trip_start.strftime('%H:%M')} - {trip_end.strftime('%H:%M')}")
        trip_count += 1
        current_time = trip_end

        # Добавить перерыв после каждых двух поездок
        if trip_count % 2 == 0:
            rest_start = current_time
            rest_end = current_time + timedelta(minutes=rest_duration)
            if rest_end > end_time:
                break

            trips.append(f"Rest: {rest_start.strftime('%H:%M')} - {rest_end.strftime('%H:%M')}")
            current_time = rest_end

    return trips


def check_continuous_coverage(intervals):
    """
    Проверяет, обеспечивает ли расписание непрерывное покрытие.
    """
    intervals.sort(key=lambda x: x[0])  # Сортировать по времени начала поездок

    for i in range(len(intervals) - 1):
        if intervals[i][1] < intervals[i + 1][0]:  # Проверить зазоры между поездками
            return False, intervals[i][1].strftime("%H:%M"), intervals[i + 1][0].strftime("%H:%M")

    return True, None, None


# Пример расписания для проверки
drivers = {
    "Driver 1": generate_schedule("00:00"),
    "Driver 2": generate_schedule("03:00"),
    "Driver 3": generate_schedule("13:15"),
    "Driver 4": generate_schedule("13:30"),
}



# Проверка непрерывного покрытия
all_intervals = []
for driver, schedule in drivers.items():
    print("\n")
    print(driver)
    for entry in schedule:
        print(entry)
        if "Trip" in entry:
            _, times = entry.split(": ")
            start, end = times.split(" - ")
            all_intervals.append((datetime.strptime(start, "%H:%M"), datetime.strptime(end, "%H:%M")))



is_covered, gap_start, gap_end = check_continuous_coverage(all_intervals)
if is_covered:
    print("Расписание обеспечивает непрерывное покрытие.")
else:
    print(f"Обнаружен разрыв между {gap_start} и {gap_end}.")




Driver 1
Trip 1: 00:00 - 01:30
Trip 2: 01:30 - 03:00
Rest: 03:00 - 03:10
Trip 3: 03:10 - 04:40
Trip 4: 04:40 - 06:10
Rest: 06:10 - 06:20
Trip 5: 06:20 - 07:55
Trip 6: 07:55 - 09:30
Rest: 09:30 - 09:40
Trip 7: 09:40 - 11:10


Driver 2
Trip 1: 03:00 - 04:30
Trip 2: 04:30 - 06:00
Rest: 06:00 - 06:10
Trip 3: 06:10 - 07:45
Trip 4: 07:45 - 09:20
Rest: 09:20 - 09:30
Trip 5: 09:30 - 11:00
Trip 6: 11:00 - 12:30
Rest: 12:30 - 12:40
Trip 7: 12:40 - 14:10


Driver 3
Trip 1: 13:15 - 14:45
Trip 2: 14:45 - 16:15
Rest: 16:15 - 16:25
Trip 3: 16:25 - 18:00
Trip 4: 18:00 - 19:35
Rest: 19:35 - 19:45
Trip 5: 19:45 - 21:15
Trip 6: 21:15 - 22:45
Rest: 22:45 - 22:55
Trip 7: 22:55 - 00:25


Driver 4
Trip 1: 13:30 - 15:00
Trip 2: 15:00 - 16:30
Rest: 16:30 - 16:40
Trip 3: 16:40 - 18:15
Trip 4: 18:15 - 19:50
Rest: 19:50 - 20:00
Trip 5: 20:00 - 21:30
Trip 6: 21:30 - 23:00
Rest: 23:00 - 23:10
Trip 7: 23:10 - 00:40
Обнаружен разрыв между 12:30 и 12:40.


In [10]:
from datetime import datetime, timedelta

# Constants
DEFAULT_WORK_DURATION = 12
DEFAULT_TRIP_DURATION = 90
DEFAULT_REST_DURATION = 10
PEAK_HOURS = [("07:00", "09:00"), ("17:00", "19:00")]
PEAK_HOURS_ADDITIONAL_TIME = 5
BUS_COUNT = 2


class Bus:
    """Represents a bus that can be assigned to a driver."""
    def __init__(self, bus_id):
        self.bus_id = bus_id
        self.is_available = True


class TripScheduler:
    def __init__(self, start_time: str, work_duration: int = DEFAULT_WORK_DURATION):
        self.start_time = datetime.strptime(start_time, "%H:%M")
        self.end_time = self.start_time + timedelta(hours=work_duration)
        self.schedule = []

    @staticmethod
    def is_peak_hour(current_time: datetime) -> bool:
        for start, end in PEAK_HOURS:
            peak_start = datetime.strptime(start, "%H:%M").time()
            peak_end = datetime.strptime(end, "%H:%M").time()
            if peak_start <= current_time.time() < peak_end:
                return True
        return False

    def get_trip_duration(self, start_time: datetime) -> int:
        trip_duration = DEFAULT_TRIP_DURATION
        end_time = start_time + timedelta(minutes=trip_duration)
        if self.is_peak_hour(start_time) or self.is_peak_hour(end_time):
            trip_duration += PEAK_HOURS_ADDITIONAL_TIME
        return trip_duration

    def generate_schedule(self):
        current_time = self.start_time
        trip_count = 0

        while current_time < self.end_time:
            trip_duration = self.get_trip_duration(current_time)
            trip_start = current_time
            trip_end = current_time + timedelta(minutes=trip_duration)
            if trip_end > self.end_time:
                break
            self.schedule.append(f"Trip {trip_count + 1}: {trip_start.strftime('%H:%M')} - {trip_end.strftime('%H:%M')}")
            trip_count += 1
            current_time = trip_end

            if trip_count % 2 == 0:
                rest_start = current_time
                rest_end = current_time + timedelta(minutes=DEFAULT_REST_DURATION)
                if rest_end > self.end_time:
                    break
                self.schedule.append(f"Rest: {rest_start.strftime('%H:%M')} - {rest_end.strftime('%H:%M')}")
                current_time = rest_end
        return self.schedule


class Driver:
    def __init__(self, name, bus: Bus):
        self.name = name
        self.bus = bus
        self.schedule = []

    def assign_schedule(self, start_time):
        scheduler = TripScheduler(start_time)
        self.schedule = scheduler.generate_schedule()


class DriverScheduler:
    def __init__(self, driver_count, bus_count):
        self.buses = [Bus(f"Bus {i + 1}") for i in range(bus_count)]
        self.drivers = [Driver(f"Driver {i + 1}", self.buses[i % bus_count]) for i in range(driver_count)]

    def find_optimal_start_times(self):
        start_increment = 1
        base_time = datetime.strptime("00:00", "%H:%M")

        for offsets in range(0, 24 * 60, start_increment):
            intervals = []
            for i, driver in enumerate(self.drivers):
                start_time = (base_time + timedelta(minutes=offsets + i * start_increment)).strftime("%H:%M")
                driver.assign_schedule(start_time)
                intervals.extend(self.parse_schedule(driver.schedule))

            is_covered, gap_start, gap_end = self.check_continuous_coverage(intervals)
            if is_covered:
                return True
        return False

    @staticmethod
    def parse_schedule(schedule):
        intervals = []
        for entry in schedule:
            if "Trip" in entry:
                _, times = entry.split(": ")
                start, end = times.split(" - ")
                intervals.append((datetime.strptime(start, "%H:%M"), datetime.strptime(end, "%H:%M")))
        return intervals

    @staticmethod
    def check_continuous_coverage(intervals):
        """Checks if the intervals ensure continuous 24-hour coverage."""
        intervals.sort(key=lambda x: x[0])  # Sort intervals by start time

        # Ensure coverage starts from 00:00 and ends at 00:00 next day
        full_start = datetime.strptime("00:00", "%H:%M")
        full_end = full_start + timedelta(days=1)

        # Check gap before the first interval
        if intervals[0][0] > full_start:
            return False, full_start.strftime("%H:%M"), intervals[0][0].strftime("%H:%M")

        # Check gaps between intervals
        for i in range(len(intervals) - 1):
            if intervals[i][1] < intervals[i + 1][0]:
                return False, intervals[i][1].strftime("%H:%M"), intervals[i + 1][0].strftime("%H:%M")

        # Check gap after the last interval
        if intervals[-1][1] < full_end:
            return False, intervals[-1][1].strftime("%H:%M"), full_end.strftime("%H:%M")

        return True, None, None


    def display_schedules(self):
        for driver in self.drivers:
            print(f"\n{driver.name} (using {driver.bus.bus_id})")
            for entry in driver.schedule:
                print(entry)


# Main program
def main():
    driver_scheduler = DriverScheduler(driver_count=4, bus_count=BUS_COUNT)
    # if driver_scheduler.find_optimal_start_times():
    #     print("\nOptimal schedules found:")
    driver_scheduler.display_schedules()
    # else:
    #     print("\nFailed to find optimal schedules.")


if __name__ == "__main__":
    main()



Driver 1 (using Bus 1)

Driver 2 (using Bus 2)

Driver 3 (using Bus 1)

Driver 4 (using Bus 2)
